In [1]:
import gym
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import CnnPolicy
from stable_baselines3.ppo.policies import MlpPolicy
from utils.adversary_env import AdversaryEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# Load target model
target_model = A2C.load("../a2c/model/lunarlander_v2_a2c_3M_to_11M")

In [2]:
# Test target model - for info on why the env is wrapped with Monitor check the evaluate_policy function
mean_reward, std_reward = evaluate_policy(target_model, Monitor(gym.make('LunarLander-v2')), n_eval_episodes=50)
print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward: 261.27 +/- 49.51


In [3]:
# Create adversary with adversary env
# adversary_env = AdversaryEnv("LunarLander-v2", target_model)
# adversary = PPO(MlpPolicy, adversary_env, verbose=1)

In [4]:
# Train adversary
# log_dir = "./out/training-log/"
# log_path = log_dir + f"adversary-lunarlander-v2/"
# save_dir = "./models/adversary-lunarlander-v2/"

# timesteps = 50000

# adversary.learn(
#         total_timesteps=timesteps,
#         callback=None,
#         log_interval=-1,
#         eval_env=adversary_env, 
#         eval_freq=5000, 
#         n_eval_episodes=100,
#         tb_log_name='PPO',
#         eval_log_path=log_path, 
#         reset_num_timesteps=False
#         )

# # Save adversary
# adversary.save(save_dir + f"adversary_lunarlander_v2_{timesteps}")

In [5]:
# Load adversary
save_dir = "./models/adversary-lunarlander-v2/"

adversary = PPO.load(save_dir + f"adversary_lunarlander_v2_50000")
adversary_env = AdversaryEnv('LunarLander-v2', target_model)

In [ ]:
import numpy as np
import pandas as pd
from uniform_attack import UniformAttack

# Evaluate uniform adversarial attack over different epsilon
data = {
    'epsilon': [],
    'mean_rew': [],
    'mean_perturbation': [],
    'mean_timesteps': [],
    'mean_n_attacks': []
}

n_episodes = 50

for j in range(1, 50):
    epsilon = j/100

    all_episodes_rewards = []
    all_episodes_perturbation = []
    all_episodes_timesteps = []
    all_episodes_n_attacks = []

    for i in range(0, n_episodes):
        # Run attack on a2c model
        env = gym.make("LunarLander-v2")
        ua = UniformAttack(env=env, model=target_model, attack=adversary, epsilon=epsilon)
        ua.perform_attack()

        all_episodes_rewards.append(ua.reward_total)
        all_episodes_perturbation.append(ua.perturbation_total)
        all_episodes_timesteps.append(ua.frames_count)
        all_episodes_n_attacks.append(ua.n_attacks)

        ua.reset_attack()

    print("-------------------------------------------------------------")
    print(f"Epsilon: {epsilon}")
    print(f"Mean reward achieved over {n_episodes} episodes: {np.mean(all_episodes_rewards):.2f}")
    print(f"Mean perturbation applied over {n_episodes} episodes: {np.mean(all_episodes_perturbation):.2f}")
    print(f"Mean number of timesteps over {n_episodes} episodes: {np.mean(all_episodes_timesteps):.2f}")
    print(f"Mean number of attacks over {n_episodes} episodes: {np.mean(all_episodes_n_attacks):.2f}")
    print("-------------------------------------------------------------")

    data['epsilon'].append(epsilon)
    data['mean_rew'].append(np.round(np.mean(all_episodes_rewards), 2))
    data['mean_perturbation'].append(np.round(np.mean(all_episodes_perturbation), 2))
    data['mean_timesteps'].append(np.round(np.mean(all_episodes_timesteps), 2))
    data['mean_n_attacks'].append(np.round(np.mean(all_episodes_n_attacks), 2))

labels = ['epsilon', 'mean_rew', 'mean_perturbation', 'mean_n_timsteps', 'mean_n_attacks']
df = pd.DataFrame(
        data, 
        columns=labels,
        )
df.set_index('epsilon', inplace=True)
with open("./out/data/" + "uniform_attack_epsilon", 'w') as f:
    df.to_csv(f)

In [9]:
from matplotlib import pyplot as plt

# Load data
with open("./out/data/" + "uniform_attack_epsilon", 'r') as f:
    df2 = pd.read_csv(f, index_col=0)

# Plot data
plt.figure(figsize=(7,9))
  plt.plot(data.index, data['mean_rew'], color='darkslategray', alpha=1, zorder=0)
  plt.vlines(data.index, ymin=0, ymax=data['mean_rew'], color=np.where(data['mean_rew'] < 0, 'maroon', 'darkgreen'), alpha=1, zorder=1)
  plt.scatter(data.index, data['mean_rew'], color=np.where(data['mean_rew'] < 0, 'maroon', 'darkgreen'), marker='D', alpha=1, zorder=2)
  plt.title("Uniform attack")
  plt.xlabel('$\epsilon$')
#   plt.ylabel()
  plt.axis([0, 1, -500, 500]) # plt.axis([xmin, xmax, ymin, ymax])
  plt.show()

[312.28, 341.94, 345.68, 273.22, 167.36, 165.8, 128.14, 106.6, 123.4, 107.66, 99.32, 95.26, 98.04, 97.26, 92.6, 90.86, 94.36, 89.72, 93.66, 90.1, 88.62, 83.66, 84.4, 82.96, 83.42, 83.5, 82.9, 86.64, 84.34, 104.82, 81.32, 82.8, 85.02, 84.7, 84.22, 82.04, 78.5, 81.38, 79.9, 78.62, 78.88, 94.38, 79.36, 93.96, 81.62, 77.56, 74.58, 77.08, 78.32]


### Todo

- Run with different epsilon
- implement strategically timed attack, etc.